In [21]:
import time
from seleniumwire import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    
opciones.add_argument('--start-maximized')         
opciones.add_argument('--incognito')
opciones.add_argument("--disable-cookies")


In [22]:
# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opciones)

url = "https://www.booking.com/searchresults.pt-br.html?ss=Pinheiros%2C+S%C3%A3o+Paulo%2C+Estado+de+S%C3%A3o+Paulo%2C+Brasil&ssne=S%C3%A3o+Paulo&ssne_untouched=S%C3%A3o+Paulo&efdco=1&label=pt-br-booking-desktop-9_uvqir24qvA6x6xGiDvCQS652796015463%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9061033%3Ali1001773%3Adec%3Adm&aid=2311236&lang=pt-br&sb=1&src_elem=sb&src=index&dest_id=2350&dest_type=district&ac_position=0&ac_click_type=b&ac_langcode=xb&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=6a8e61d6955d032f&ac_meta=GhA2YThlNjFkNjk1NWQwMzJmIAAoATICeGI6CXBpbmhlaXJvc0AASgBQAA%3D%3D&checkin=2023-09-12&checkout=2023-09-13&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"
driver.get(url)

In [23]:
wait = WebDriverWait(driver, 20)

try:
    cookie_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler")))
    cookie_button.click()
    
    
except TimeoutException:
    pass  

In [24]:
button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="header-currency-picker-trigger"]')
button.click()

In [25]:
button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="selection-item"]')
button.click()

In [26]:
try:
    
    button_element = driver.find_element(By.XPATH, "//button[@aria-label='Ignorar informações de login.']")
    button_element.click()
except NoSuchElementException:
    pass

In [42]:
properties = []


def extract_properties():
    property_cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')
    for property in property_cards:
        name = property.find_element(By.CSS_SELECTOR, 'div[data-testid="title"]').text
        price = property.find_element(By.CSS_SELECTOR, '[data-testid="price-and-discounted-price"]').text
        chk_out = property.find_element(By.XPATH, '//button[contains(@class, "d47738b911") and @data-testid="date-display-field-end"]').text
        
        new_property = {}
        new_property['name'] = name
        new_property['price'] = price
        new_property['chk_out'] = chk_out
        
        properties.append(new_property)


total_pages = int(driver.find_element(By.CSS_SELECTOR, 'div[data-testid="pagination"] li:last-child').text)
print(f"Total pages: {total_pages}")

current_page = 0

Total pages: 4


In [43]:
for current_page in range(total_pages):
    driver.delete_all_cookies()
    extract_properties()
    next_page_btn = driver.find_element(By.CSS_SELECTOR, 'button[aria-label*="Página seguinte"]')
    next_page_btn.click()

In [44]:
print(properties)

[{'name': 'Charlie Alves Guimarães Pinheiros - Soft opening', 'price': 'R$ 485', 'chk_out': 'qua., 13 de set.'}, {'name': 'Uniclass Hotel Pinheiros', 'price': 'R$ 417', 'chk_out': 'qua., 13 de set.'}, {'name': 'Charlie Faria Lima', 'price': 'R$ 517', 'chk_out': 'qua., 13 de set.'}, {'name': 'Studio Quality Oscar Freire', 'price': 'R$ 307', 'chk_out': 'qua., 13 de set.'}, {'name': 'Charlie Rua dos Pinheiros', 'price': 'R$ 435', 'chk_out': 'qua., 13 de set.'}, {'name': 'Ô de Casa Hostel', 'price': 'R$ 209', 'chk_out': 'qua., 13 de set.'}, {'name': 'Radisson Pinheiros', 'price': 'R$ 1.307', 'chk_out': 'qua., 13 de set.'}, {'name': 'Q-Flats Hotel - Faria Lima', 'price': 'R$ 419', 'chk_out': 'qua., 13 de set.'}, {'name': 'Guest Urban Hotel Design Pinheiros', 'price': 'R$ 437', 'chk_out': 'qua., 13 de set.'}, {'name': 'Casa Savana', 'price': 'R$ 360', 'chk_out': 'qua., 13 de set.'}, {'name': 'Filadelfia Hotel', 'price': 'R$ 218', 'chk_out': 'qua., 13 de set.'}, {'name': 'HomeLike ao lado da 

In [45]:
import pandas as pd
df = pd.DataFrame(properties)
df.head()

,name,price,chk_out
0,Charlie Alves Guimarães Pinheiros - Soft opening,R$ 485,"qua., 13 de set."
1,Uniclass Hotel Pinheiros,R$ 417,"qua., 13 de set."
2,Charlie Faria Lima,R$ 517,"qua., 13 de set."
3,Studio Quality Oscar Freire,R$ 307,"qua., 13 de set."
4,Charlie Rua dos Pinheiros,R$ 435,"qua., 13 de set."


In [46]:
df.shape

(88, 3)

In [47]:
df.info

<bound method DataFrame.info of                                                  name   price  \
0    Charlie Alves Guimarães Pinheiros - Soft opening  R$ 485   
1                            Uniclass Hotel Pinheiros  R$ 417   
2                                  Charlie Faria Lima  R$ 517   
3                         Studio Quality Oscar Freire  R$ 307   
4                           Charlie Rua dos Pinheiros  R$ 435   
..                                                ...     ...   
83                     Hotel Pousada Clínicas Valente  R$ 202   
84                                         Hotel Bali  R$ 240   
85                            Pinheiros Hostel & Food   R$ 96   
86  Hostel Calábria 5 minutos da Vila Madalena e B...  R$ 187   
87              Vilarejo Hostel Pinheiros Guest House  R$ 145   

             chk_out  
0   qua., 13 de set.  
1   qua., 13 de set.  
2   qua., 13 de set.  
3   qua., 13 de set.  
4   qua., 13 de set.  
..               ...  
83  qua., 13 de set.  
84 

In [48]:
name_counts = df['name'].value_counts()
print(name_counts)

name
Charlie Alves Guimarães Pinheiros - Soft opening          2
Golden Tower Pinheiros by Fênix Hotéis                    2
Studio no Jardins proximo Metro Oscar Freire              2
Uniclass Hotel Pinheiros                                  2
Comfort Flat Pinheiros em Hotel 4,5 estrelas              2
                                                         ..
360 Suítes VN Capote Valente - Apartamentos mobiliados    1
Loft Moderno na Rua Oscar Freire - Jardins                1
Flat Lux Mercure in Pinheiros                             1
Duplex de Luxo Oscar Freire                               1
Vilarejo Hostel Pinheiros Guest House                     1
Name: count, Length: 62, dtype: int64


In [49]:
# https://www.booking.com/searchresults.pt-br.html?ss=Jardins%2C+S%C3%A3o+Paulo%2C+Estado+de+S%C3%A3o+Paulo%2C+Brasil&ssne=Pinheiros&ssne_untouched=Pinheiros&label=pt-br-booking-desktop-9_uvqir24qvA6x6xGiDvCQS652796015463%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9061033%3Ali1001773%3Adec%3Adm&aid=2311236&lang=pt-br&sb=1&src_elem=sb&src=searchresults&dest_id=1805&dest_type=district&ac_position=2&ac_click_type=b&ac_langcode=xb&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=d2b9623de6ea0191&ac_meta=GhBkMmI5NjIzZGU2ZWEwMTkxIAIoATICeGI6B2phcmRpbnNAAEoAUAA%3D&checkin=2023-09-12&checkout=2023-09-13&group_adults=2&no_rooms=1&group_children=0

In [50]:
driver.quit()

In [51]:
df.to_csv('pinheiros.csv', index=False)